In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import asyncio
import os
import time
import pickle
import faiss
import numpy as np
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import webbrowser
from contextlib import contextmanager
from agents import Agent, Runner, handoff, function_tool
from agents.handoffs import HandoffInputData
from prompt import (
    MANAGER_INSTRUCTION,
    PRODUCT_INSTRUCTION,
    SHOP_INFORMATION_INSTRUCTION
)
from rag import product_rag, get_shop_information

# === Bypass OpenAI trace
@contextmanager
def trace(*args, **kwargs):
    yield

# === FAISS config
INDEX_PATH = "faiss_mobile_shop.index"
META_PATH = "faiss_mobile_shop_metadata.pkl"
DIM = 384

load_dotenv()
model = SentenceTransformer("all-MiniLM-L6-v2")

# === FAISS setup
if os.path.exists(INDEX_PATH):
    index = faiss.read_index(INDEX_PATH)
    with open(META_PATH, "rb") as f:
        metadatas = pickle.load(f)
else:
    index = faiss.IndexFlatIP(DIM)
    metadatas = []

if index.d != DIM:
    print(f"⚠️ FAISS index dimension {index.d}, model {DIM}. Reset!")
    index = faiss.IndexFlatIP(DIM)
    metadatas = []

if index.ntotal == 0:
    print("⚠